In [56]:
name="小王"
month="三月"
print("{}在{}的工资为{}".format(name, month,5000))

小王在三月的工资为5000


In [61]:

import torch.nn.functional as F
def load_model(model_path, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torch.load(model_path, map_location=device)
    return model.to(device)
def prepare_input(device, batch_size=1, channels=3, height=640, width=640):
    x = torch.randn(batch_size, channels, height, width, device=device)
    return x
def infer(model_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 加载模型
    model = load_model(model_path, device)
    model.eval()  # 设置为评估模式
    
    # 准备输入数据
    x = prepare_input(device)
    
    # 执行前向传播
    with torch.no_grad():
        output = model(x)
    
    # 打印输出的形状（作为示例）
    print(output.size())

# 调用推理函数
model_path = 'best.pt'  # 确保这是你的模型文件的正确路径
infer(model_path)


AttributeError: 'dict' object has no attribute 'to'

In [1]:
import cv2

for i in range(100):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print('Webcam found at index: ' + str(i))
        cap.release()

Webcam found at index: 0
Webcam found at index: 1
Webcam found at index: 2


In [ ]:
import cv2
 
video = cv2.VideoCapture("rtmp://localhost:1935/live/123")
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)
size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print(size)
while True:
    ret, frame = video.read()
    cv2.imshow("A video", frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
video.release()
cv2.destroyAllWindows()

In [ ]:
import face_recognition

# 加载第一张照片并识别面部特征
image1 = face_recognition.load_image_file("C:/Users/wsh/Pictures/Camera Roll/f1.jpg")
encoding1 = face_recognition.face_encodings(image1)[0]

# 加载第二张照片并识别面部特征
image2 = face_recognition.load_image_file("C:/Users/wsh/Pictures/Camera Roll/f2.jpg")
encoding2 = face_recognition.face_encodings(image2)[0]

# 比较两个面部特征
results = face_recognition.compare_faces([encoding1], encoding2)

if results[0]:
    print("这两张照片是同一个人。")
else:
    print("这两张照片不是同一个人。")

In [29]:
import io
import pstats
import time
from pyexpat import model
import cv2
import numpy as np
import torch
from configList import *
from utils.augmentations import letterbox
from utils.general import check_img_size, non_max_suppression, scale_coords
from utils.plots import Annotator
WEIGHTS = 'weights/yolov5n.pt'
IMGSZ = [640, 640]  # 图像尺寸
CONF_THRES = 0.5  # 置信度阈值
IOU_THRES = 0.5  # IOU阈值
MAX_DET = 1000  # 最大检测数量
LINE_THICKNESS = 1  # 线条厚度
HIDE_CONF = False  # 是否隐藏置信度
HIDE_LABELS = None  # 是否隐藏标签
model, device, half, stride, names = get_model()
imgsz = check_img_size([640, 640], s=stride)
def colors(index, bright=True):
    color_list = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]
    return color_list[index % len(color_list)]
from models.experimental import attempt_load
from utils.torch_utils import select_device

WEIGHTS = "D:/Smoke/SmokeDetectionMasterPython/quantized_model.pt"
# WEIGHTS = "D:/BaiduNetdiskDownload/smoking_calling_weight/exp2052/weights/best.pt"


def get_model():
    device = select_device('')
    half = device.type != 'cpu'
    model = attempt_load(WEIGHTS, map_location=device)
    stride = int(model.stride.max())
    names = model.module.names if hasattr(model, 'module') else model.names
    if half:
        model.half()
    return model, device, half, stride, names
def pred_img_optimized_async(img0, model, device, imgsz, names, conf_thres, iou_thres, half,
                                   line_thickness, hide_labels, hide_conf, max_det):
        img = letterbox(img0, new_shape=imgsz, auto=True)[0]
        img = img.transpose((2, 0, 1))[::-1]
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()
        img /= 255.0  # 归一化
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        im0 = img0.copy()
        annotator = Annotator(im0, line_width=line_thickness, example=str(names))

        pred = model(img, augment=False, visualize=False)[0]
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False, max_det=max_det)

        detections = []
        det = pred[0]
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                c = int(cls)
                detections.append({'coords': tuple(map(int, xyxy)), 'confidence': conf.item()})

                label = None if hide_labels else f'{names[c]} {conf:.2f}' if not hide_conf else names[c]
                annotator.box_label(xyxy, label, color=colors(c))

        return im0, detections
cap = cv2.VideoCapture(0)

# 用于计算FPS的变量
frame_count = 0
start_time = time.time()
def fx():
    global frame_count, start_time
    start_time = time.time()
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 对当前帧进行处理和预测
        img, detections = pred_img_optimized_async(frame, model, device, IMGSZ, names, CONF_THRES,
                                                    IOU_THRES, half, LINE_THICKNESS, HIDE_LABELS, HIDE_CONF,
                                                    MAX_DET)
        if detections:
            print("抽了")
        
        # 显示处理后的图像
        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time
        ret, buffer = cv2.imencode('.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), 70])
        if not ret:
            print("Failed to encode frame")

        if elapsed_time >= 20: 
            print("终止")# 运行大约30秒后中断
            return 


if __name__ == "__main__":
    import cProfile
    profiler = cProfile.Profile()
    profiler.enable()
    fx()
    profiler.disable()
    print("============")
    
    # 保存分析结果到文件
    stats = pstats.Stats(profiler).sort_stats('cumtime')
    stats.dump_stats('profiling_results.prof')
    
    # 导出为csv
    s = io.StringIO()
    stats = pstats.Stats('profiling_results.prof', stream=s)
    stats.sort_stats('cumtime')
    stats.print_stats()
    
    # 获取性能数据作为CSV格式
    profiling_data = s.getvalue()
    # 将数据转换为CSV格式
    profiling_data = "ncalls" + profiling_data.split("ncalls")[-1]
    profiling_data = '\n'.join([','.join(line.rstrip().split(None, 5)) for line in profiling_data.split('\n')])
    
    # 将CSV数据保存到文件
    with open('profiling_results.csv', 'w') as f:
        f.write(profiling_data)
    cap.release()
    # cv2.destroyAllWindows()

YOLOv5  edf1a57 torch 2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)

Fusing layers... 
YOLOv5l summary: 400 layers, 47370047 parameters, 0 gradients


抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
抽了
终止


In [31]:
import pandas as pd

# 读取CSV
df = pd.read_csv('profiling_results.csv')
sorted_df = df[['cumtime', 'filename:lineno(function)']].sort_values(by='cumtime', ascending=False)

sorted_df

,cumtime,filename:lineno(function)
0,20.014,C:\Users\wsh\AppData\Local\Temp\ipykernel_2337...
1,13.645,C:\Users\wsh\AppData\Local\Temp\ipykernel_2337...
2,7.156,D:\Smoke\SmokeDetectionMasterPython\utils\gene...
3,5.880,D:\conda_home\envs\yolov5-master\lib\site-pack...
4,5.878,D:\conda_home\envs\yolov5-master\lib\site-pack...
...,...,...
145,0.000,{built-in method torch.meshgrid}
146,0.000,{method 'startswith' of 'str' objects}
147,0.000,{built-in method nt.getpid}
148,0.000,D:\conda_home\envs\yolov5-master\lib\site-pack...
